In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import statsmodels.api as sm
import pickle
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')

In [2]:
ts_df = pd.read_csv('./data/SWOW_prediction/eval/time_series/nyt_ts_df.csv')
ts_df = ts_df.groupby(['words','year','property']).outputs_z.mean().reset_index()

# Gallup moral issues


In [ ]:
import os
os.listdir('./data/Gallup')

['computer.csv',
 'gun_ownership.csv',
 'sex.csv',
 'worry_environment.csv',
 'environment_quality.csv',
 'real_estate.csv',
 'worry_drug.csv',
 'gay_relation.csv',
 'abortion_important_to_vote.csv',
 'nuclear_energy_use.csv',
 'animal_medical_testing.csv',
 'movies.csv',
 'gambling.csv',
 'stem_cell.csv',
 'birth_control.csv',
 'education.csv',
 'polygamy.csv',
 'publishing.csv',
 'afghanistan.csv',
 'grocery.csv',
 'banking.csv',
 'marijuana_legal.csv',
 'animal_cloning.csv',
 'gun_possession.csv',
 'pharmaceutical.csv',
 'worry_terrorism.csv',
 'abortion_legal.csv',
 'divorce.csv',
 'affair.csv',
 'samesex.csv',
 'worry_healthcare.csv',
 'euthanasia.csv',
 'human_cloning.csv',
 'pornography.csv',
 'healthcare.csv',
 'retail.csv',
 'accounting.csv',
 'worry_air_pollution.csv',
 'telephone.csv',
 'oil_gas.csv',
 'federal_government.csv',
 'abortion.csv',
 'sports.csv',
 'gun_laws_stricter.csv',
 'worry_ozone.csv',
 'death_penalty.csv',
 'internet.csv',
 'airline.csv',
 'television.csv

In [4]:
issues = ['abortion', 'animal_cloning', 'human_cloning', 
          'animal_medical_testing',
          'sex', 'birth_control','death_penalty','divorce',
         'samesex','fur','polygamy','pornography','stem_cell',
          'suicide','gambling','euthanasia','affair'
         ]

In [5]:
dfs= {issue: pd.read_csv(f'./data/Gallup/{issue}.csv') for issue in issues}

In [6]:
for issue, df in dfs.items():
    columns = ['Morally acceptable', 'Morally wrong', 'Not a moral issue (vol.)']
    signs = [1, -1, 0]
    df = df.fillna("0%")
    df = df.loc[df.Geography != '0%'] #Removing empty lines
    for column in columns:
        
        df[column] = df[column].apply(lambda x : float(x.replace('%','')))
    df['total_vote'] = df[columns].sum(axis = 1)
    polarity_votes = []
    moral_votes = []
    for i, row in df.iterrows():
        polarity_vote = 0
        moral_vote = 0
        # polarity_vote = row['Morally acceptable'] / row['total_vote']
        
        for column, sign in zip(columns,signs):
            polarity_vote += row[column] * sign
            moral_vote += row[column] * abs(sign)

        moral_vote /= row['total_vote']
        polarity_vote /= row['total_vote']
        polarity_votes.append(polarity_vote)
        moral_votes.append(moral_vote)
    
    df['polarity_vote'] = polarity_votes
    df['morality_vote'] = moral_votes
    dfs[issue] = df
    
        

In [7]:
token_mapping = {'abortion':['abortion'],
                 'animal_cloning':['animal cloning'],
                 'human_cloning': ['human cloning'],
                 'birth_control': ['birth control','contraceptive','contraception'],
                 'death_penalty': ['death penalty'],
                 'animal_medical_testing': ['animal testing'],
                 'divorce': ['divorce'],
                'samesex':['homosexuality','homosexual'],
                'fur':['fur'],
                 'sex':['sex'],
                'polygamy': ['polygamy'],
                'pornography': ['pornography','porn'],
                'stem_cell': ['stem cell'],
                'suicide': ['suicide'],
                'gambling':['gambling'],
                'euthanasia': ['euthanasia'],
                'affair': ['adultery']}


In [8]:
total_df = pd.DataFrame()
for issue in issues:
    issue_df = dfs[issue]
    tokens = token_mapping[issue]
    token_df = ts_df.loc[ts_df.words.isin(tokens)].reset_index()
    moral_votes = []
    polarity_votes = []
    for i, row in token_df.iterrows():
        year = row['year']
        moral_vote = issue_df.loc[issue_df.Time == year]
        if len(moral_vote) > 0:
            moral_votes.append(moral_vote.iloc[0]['morality_vote'])
            polarity_votes.append(moral_vote.iloc[0]['polarity_vote'])
        else:
            moral_votes.append(None)
            polarity_votes.append(None)
    
    token_df['morality_vote'] =  moral_votes
    token_df['polarity_vote'] = polarity_votes
    token_df['issue'] = [issue] * len(token_df)
    total_df = pd.concat((total_df, token_df), ignore_index = True)
        
    

In [9]:
total_df = total_df.loc[~pd.isna(total_df.morality_vote)]

In [10]:
total_df.words.unique()

array(['abortion', 'sex', 'death penalty', 'divorce', 'homosexual',
       'homosexuality', 'fur', 'stem cell', 'suicide', 'gambling',
       'adultery'], dtype=object)

In [11]:
total_df = total_df.groupby(['issue','year','property','words'])[['outputs_z','morality_vote','polarity_vote']].mean().reset_index()

In [12]:
issues = total_df.issue.unique()
list_rows = []
for i, row in total_df.iterrows():
    issue = row['issue']
    year = row['year']
    p = row['property']
    word = row['words']
    
    new_df = total_df.loc[(total_df.issue == issue) & (total_df.property == p) & (total_df.year == year + 1)]
    if len(new_df) > 0:
        polarity_df = new_df.copy(deep = True)
        p1 = row['outputs_z']
        p2 = polarity_df.outputs_z.mean()
        pol_vote_1 = row['polarity_vote']
        pol_vote_2 = polarity_df.polarity_vote.mean()

        morality_vote_1 = row['morality_vote']
        morality_vote_2 = polarity_df.morality_vote.mean()


        list_rows.append({
            'words':word,
            'property':p,
            'p1':p1,
            'p2':p2,
            'p_change': p2 - p1,
            'pol_vote_1':pol_vote_1,
            'pol_vote_2':pol_vote_2,
            'pol_vote_change':(pol_vote_2 - pol_vote_1),
            'morality_vote_1':morality_vote_1,
            'morality_vote_2':morality_vote_2,
            'morality_vote_change':(morality_vote_2 - morality_vote_1),
            'year1':year,
            'year2':year + 1,
            'issue': issue,
        })
rate_df = pd.DataFrame(list_rows)

In [13]:

rate_df['p_change_z'] = (rate_df['p_change'] - rate_df['p_change'].mean()) / rate_df['p_change'].std()


rate_df['pol_vote_change_z'] = (rate_df['pol_vote_change'] - rate_df['pol_vote_change'].mean()) / rate_df['pol_vote_change'].std()
rate_df['morality_vote_change_z'] = (rate_df['morality_vote_change'] - rate_df['morality_vote_change'].mean()) / rate_df['morality_vote_change'].std()

#Normalize morality_vote and polarity_vote
rate_df['pol_vote_1'] = (rate_df['pol_vote_1'] - rate_df['pol_vote_1'].mean()) / rate_df['pol_vote_1'].std()
rate_df['pol_vote_2'] = (rate_df['pol_vote_2'] - rate_df['pol_vote_2'].mean()) / rate_df['pol_vote_2'].std()
rate_df['morality_vote_1'] = (rate_df['morality_vote_1'] - rate_df['morality_vote_1'].mean()) / rate_df['morality_vote_1'].std()
rate_df['morality_vote_2'] = (rate_df['morality_vote_2'] - rate_df['morality_vote_2'].mean()) / rate_df['morality_vote_2'].std()


In [14]:
rate_df = rate_df.dropna().reset_index(drop = True)

In [15]:
rate_df_relevance = rate_df.loc[rate_df.property == 'previous_link']
rate_df_polarity = rate_df.loc[rate_df.property == 'polarity']

indices = ['words','pol_vote_1','pol_vote_2','pol_vote_change_z','pol_vote_change','morality_vote_1','morality_vote_2','morality_vote_change_z','morality_vote_change','issue','year1','year2']
rate_df = rate_df_relevance.set_index(indices).\
join(
    rate_df_polarity.set_index(indices),
    lsuffix = '_relevance', rsuffix = '_polarity'
)



In [16]:
rate_df.reset_index(inplace = True)

In [17]:
rate_df.sample()

,words,pol_vote_1,pol_vote_2,pol_vote_change_z,pol_vote_change,morality_vote_1,morality_vote_2,morality_vote_change_z,morality_vote_change,issue,...,property_relevance,p1_relevance,p2_relevance,p_change_relevance,p_change_z_relevance,property_polarity,p1_polarity,p2_polarity,p_change_polarity,p_change_z_polarity
3,abortion,-0.241167,-0.276435,-0.311741,-0.009768,0.724535,0.760075,-0.051166,0.0,abortion,...,previous_link,0.981614,0.956254,-0.02536,-0.218495,polarity,-0.070877,-0.091917,-0.02104,-0.186185


In [18]:
formula = 'pol_vote_change ~ year1  + C(issue) + p_change_relevance + p_change_polarity + pol_vote_1'
model = smf.ols(formula, data=rate_df).fit() 
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        pol_vote_change   R-squared:                       0.485
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     2.896
Date:                Sat, 11 Jan 2025   Prob (F-statistic):            0.00491
Time:                        11:45:01   Log-Likelihood:                 98.640
No. Observations:                  54   AIC:                            -169.3
Df Residuals:                      40   BIC:                            -141.4
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   -22.6298      9.213     -2.456      0.018     -41.251      -4.009
C(issue)[T.affair]           -0.6186      0.120     -5.159      0.000      -0.861      -0.376
C(issue)[T.death_penalty]     0.4521      0.085      5.339      0.000       0.281       0.623
C(issue)[T.divorce]           0.4504      0.083      5.415      0.000       0.282       0.619
C(issue)[T.fur]               0.3286      0.067      4.935      0.000       0.194       0.463
C(issue)[T.gambling]          0.3584      0.069      5.185      0.000       0.219       0.498
C(issue)[T.samesex]           0.0492      0.033      1.488      0.145      -0.018       0.116
C(issue)[T.sex]               0.2645      0.050      5.267      0.000       0.163       0.366
C(issue)[T.stem_cell]         0.3267      0.057      5.753      0.000       0.212       0.441
C(issue)[T.suicide]          -0.4771      0.096     -4.965      0.000      -0.671      -0.283
year1                         0.0112      0.005      2.448      0.019       0.002       0.021
p_change_relevance           -0.0346      0.045     -0.764      0.449      -0.126       0.057
p_change_polarity             0.1199      0.054      2.215      0.033       0.010       0.229
pol_vote_1                   -0.3770      0.070     -5.390      0.000      -0.518      -0.236
==============================================================================
Omnibus:                        0.967   Durbin-Watson:                   2.402
Prob(Omnibus):                  0.617   Jarque-Bera (JB):                0.374
Skew:                          -0.137   Prob(JB):                        0.829
Kurtosis:                       3.302   Cond. No.                     3.00e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  3e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
formula = 'pol_vote_change ~ year1  + C(issue) + pol_vote_1'
model = smf.ols(formula, data=rate_df).fit() 
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        pol_vote_change   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     2.754
Date:                Sat, 11 Jan 2025   Prob (F-statistic):            0.00885
Time:                        11:45:06   Log-Likelihood:                 95.397
No. Observations:                  54   AIC:                            -166.8
Df Residuals:                      42   BIC:                            -142.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   -16.4875      9.108     -1.810      0.077     -34.867       1.892
C(issue)[T.affair]           -0.5593      0.121     -4.635      0.000      -0.803      -0.316
C(issue)[T.death_penalty]     0.4071      0.085      4.777      0.000       0.235       0.579
C(issue)[T.divorce]           0.4092      0.084      4.869      0.000       0.240       0.579
C(issue)[T.fur]               0.2954      0.067      4.392      0.000       0.160       0.431
C(issue)[T.gambling]          0.3248      0.070      4.647      0.000       0.184       0.466
C(issue)[T.samesex]           0.0397      0.034      1.168      0.250      -0.029       0.108
C(issue)[T.sex]               0.2373      0.051      4.698      0.000       0.135       0.339
C(issue)[T.stem_cell]         0.2984      0.057      5.200      0.000       0.183       0.414
C(issue)[T.suicide]          -0.4251      0.096     -4.413      0.000      -0.620      -0.231
year1                         0.0082      0.005      1.802      0.079      -0.001       0.017
pol_vote_1                   -0.3397      0.070     -4.833      0.000      -0.482      -0.198
==============================================================================
Omnibus:                        0.095   Durbin-Watson:                   2.320
Prob(Omnibus):                  0.954   Jarque-Bera (JB):                0.190
Skew:                           0.093   Prob(JB):                        0.909
Kurtosis:                       2.777   Cond. No.                     2.86e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.86e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
token_mapping

{'abortion': ['abortion'],
 'animal_cloning': ['animal cloning'],
 'human_cloning': ['human cloning'],
 'birth_control': ['birth control', 'contraceptive', 'contraception'],
 'death_penalty': ['death penalty'],
 'animal_medical_testing': ['animal testing'],
 'divorce': ['divorce'],
 'samesex': ['homosexuality', 'homosexual'],
 'fur': ['fur'],
 'sex': ['sex'],
 'polygamy': ['polygamy'],
 'pornography': ['pornography', 'porn'],
 'stem_cell': ['stem cell'],
 'suicide': ['suicide'],
 'gambling': ['gambling'],
 'euthanasia': ['euthanasia'],
 'affair': ['adultery']}